In [1]:
import torch

from model import RetNET
from tinystories import Task

from functools import partial

In [2]:
iters = partial(Task.iter_batches, batch_size = 32, max_seq_len = 256, device = 'cuda', num_workers = 4)

In [3]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        batch_iter = iters(split)
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = next(batch_iter)
            _, loss, _ = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [4]:
class ModelConifg:
    nembed = 288
    nheads = 6
    nlayers = 6

    vocab_size = 32000
    block_size = 256

    device = 'cuda'

In [5]:
config = ModelConifg

In [6]:
model = RetNET(config).to('cuda')
print("Parametres", sum(p.numel() for p in model.parameters()))

Parametres 22951904


In [7]:
max_iters = 5000
eval_iters = 50
optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4)

train_iters = iters('train')
xb, yb = next(train_iters)

from tqdm import tqdm
pbar = tqdm(range(max_iters))

micro_batches = 2

for i in pbar:
    for m in range(micro_batches):
        _, loss, _ = model(xb, yb)
        loss = loss / micro_batches
        xb, yb = next(train_iters) # to retrieve batches asyncally on cpu while model in doing its ting in gpu
        loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none = True)
    # pbar.set_description(f"Loss {loss.item():.4f}")
    if i % 500== 0:
        losses = estimate_loss()
        print(f"\nStep {i}: train_loss {losses['train']:.4f}  val_loss {losses['val']:.4f}")

  0%|          | 0/5000 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [8]:
model.load_state_dict(torch.load("tiny_stories_28866.pt"))

<All keys matched successfully>

In [9]:
from tokenizer import Tokeniser

In [10]:
enc = Tokeniser()

In [19]:
context = "Kyle"
out = model.generate(torch.tensor([enc.encode(context, True, False)], dtype = torch.long, device = 'cuda'), steps = 300)
print("".join(enc.decode(out[0].tolist())))

Kyle was very excited to keep everyone safe. One day they went for a drive in their car. As they drove by, they found a beautiful park. In the park was a slide, windows with a rainbow slide. They all laughed and played there too.
As the day went on, Kat and their moms let them go in. Everyone was so happy and excited!
Felix Celia and their mum would always be ready for all of the animals. They raced around through the park so they could see even small birds in the sky while they drove away.
They packed up their carrots and then they returned to their destination.
When they arrived, everyone was so excited. They were so excited. They explored the park and finally arrived in the park. It was a beautiful place in the park.
"Look at this expensive thing!" Rachel said proudly.
Welia smiled and they enjoyed the cool trash of their adventure. Billy and her mommy were playing in the park together. Billy was having so much fun, jumping around and swinging in the trees. Suddenly, Billy's mom cam